In [1]:
# import modules
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import io
import googleapiclient.http

from PIL import Image,ImageDraw,ImageFont
import os

import numpy as np
import pandas as pd
import re

import img2pdf

import tkinter as tk
from tkinter import ttk
from datetime import datetime
from tkinter import filedialog

In [3]:
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

drive_service = build('drive', 'v3', credentials=creds)

In [4]:
# 품명/수량/금액 나누는 메소드
def divide(full):
    item = full.split('\n')
    item_list = [i.split("/") for i in item]
    for i in range(len(item_list)):
        item_list[i][2] = str(commaParse(item_list[i][2]))+" 원"
    return item_list

# 사진 링크에서 id 뽑아내는 메소드
def get_id(url_list):
    url_list = [url+";" for url in url_list]
    pattern = re.compile('{}(.*?){}'.format(re.escape('id='), re.escape(';')))
    return [pattern.findall(url) for url in url_list] 

# 숫자를 000,000,000 꼴로 바꾸는 정규식
def commaParse(num):
    return re.sub('(?<=\d)(?=(\d{3})+(?!\d))',',',str(num))

In [23]:
class Receipt:
    def __init__(self, filepath):
        self.filetype = filepath.split('.')[-1]
        if self.filetype == 'csv':
            self.raw = pd.read_csv(filepath)
        else:
            self.raw = pd.read_excel(filepath)
    
    def make_use_list(self):
        df = self.raw.copy()

        df['영수증'] = self.get_id(df['영수증 (사진 파일로 업로드)'])
        df['개수'] = [len(v) for v in df['영수증']]
        df = df.drop('영수증 (사진 파일로 업로드)', axis=1)
        self.real_receipt = sum(df['개수'][df['실물영수증 여부']=='실물영수증 O'])

        df = df[['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수']]
        self.use_list_df = pd.DataFrame(columns=['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수'])

        index = 0
        for idx, val in df.iterrows():
            num = val['개수']
            for i in range(num):
                self.use_list_df.loc[index] = val
                index +=1
                
    def save_use_list(self, filepath):
        self.use_list_df.to_excel(f'{filepath}/예산사용내역서')
        
    def make_receipt_df(self):
        df = self.raw.copy()

        # 필요한 column만 남기기
        df = df[['사용 RA','인원','결제일자','프로그램명','금액', '업체선정사유', '품명/수량/금액','영수증 (사진 파일로 업로드)']]

        # id만 list로 뽑아내는 과정
        df['영수증'] = get_id(df['영수증 (사진 파일로 업로드)'])
        df = df.drop('영수증 (사진 파일로 업로드)', axis=1)

        # 원하는 꼴로 변형
        df['인원'] = [str(v)+" 명" for v in df['인원']]
        df['금액'] = [str(self.make_comma(v))+" 원" for v in df['금액']]
        
        self.receipt_df = df

    def save_receipt(self, row, num, filepath):
        loc_short = [(1640, 205), (2600, 205), (690,270), (1600, 270),(2555, 270), (550,525)]
        loc_long = [(1640, 205), (2600, 205), (690,270), (1550, 270),(2555, 270), (550,525)]
        loc_house = (690, 205)
        loc_image = (300,670)
        col_item = [2260, 2690, 2782]
        row_item = [685, 830, 964, 1107, 1265, 1402, 1540, 1677, 1830, 1962]

        # 폰트 경로와 사이즈 설정
        regularFont =ImageFont.truetype('fonts/HANBatangB.ttf',36)
        smallFont =ImageFont.truetype('fonts/HANBatangB.ttf',30)

        target_image = Image.open("form.jpeg")
        row = df.iloc[index]
        draw = ImageDraw.Draw(target_image)

        # 하우스명 기입
        draw.text(loc_house, "윤동주 하우스", fill="black", font=regularFont, align='center')

        # 사용RA, 인원, 결제일자, 프로그램명, 금액, 업체선정사유 기입
        if (len(row[3]) > 9):
            for i in range(6):
                draw.text(loc_long[i], str(row[i]), fill="black", font=regularFont, align='center')
        else:
            for i in range(6):
                draw.text(loc_short[i], str(row[i]), fill="black", font=regularFont, align='center')

        # 품명 기입
        if type(row[6]) == type('string'):
            item_list = self.divide_item(row[6])
            for i in range(len(item_list)):
                # 품명
                if len(item_list[i][0]) > 13: # 품명 긴 경우
                    front = item_list[i][0][:13]
                    back = item_list[i][0][13:]
                    draw.text((col_item[0], row_item[i]-20), front, fill="black", font=smallFont, align='center') 
                    draw.text((col_item[0], row_item[i]+20), back, fill="black", font=smallFont, align='center') 
                else: # 품명 짧은 경우
                    draw.text((col_item[0], row_item[i]), item_list[i][0], fill="black", font=smallFont, align='center') 
                draw.text((col_item[1], row_item[i]), item_list[i][1], fill="black", font=smallFont, align='center') # 수량
                draw.text((col_item[2], row_item[i]), item_list[i][2], fill="black", font=smallFont, align='center') # 금액

        # 영수증 사진
        check = 1
        for i in range(len(row[7])):
            check += 1
            fh = io.BytesIO()
            file_id = row[7][i]
            request = drive_service.files().get_media(fileId=file_id)
            downloader = googleapiclient.http.MediaIoBaseDownload(fh, request)

            done = False
            while done is False:
                status, done = downloader.next_chunk()

            add_image = Image.open(fh)
            if int(add_image.size[0]*(1300/add_image.size[1])) > 1300:
                target_image.paste(im = add_image.resize((1300, int(add_image.size[1]*(1300/add_image.size[0])))), box=loc_image)              
            else:
                target_image.paste(im = add_image.resize((int(add_image.size[0]*(1300/add_image.size[1])), 1300)), box=loc_image)
            
            target_image.save(f"{filepath}/{num}_{row[3]}_{check}.png")
            
        # 품명/수량/금액 나누는 메소드
        def divide_item(self, full):
            item = full.split('\n')
            item_list = [i.split("/") for i in item]
            for i in range(len(item_list)):
                item_list[i][2] = str(self.make_comma(item_list[i][2]))+" 원"
            return item_list

        # 사진 링크에서 id 뽑아내는 메소드
        def get_id(self, url_list):
            url_list = [url+";" for url in url_list]
            pattern = re.compile('{}(.*?){}'.format(re.escape('id='), re.escape(';')))
            return [pattern.findall(url) for url in url_list] 

        # 숫자를 000,000,000 꼴로 바꾸는 정규식
        def make_comma(self, num):
            return re.sub('(?<=\d)(?=(\d{3})+(?!\d))',',',str(num))

In [ ]:
def get_folder():
    folder = filedialog.askdirectory(title="저장할 위치를 선택하세요.")
    return folder

def get_file():
    file = filedialog.askopenfilename(title="파일을 선택하세요.", filetypes=(("csv files", "*.csv"), ("xlsx files", "*.xlsx")))   
    return file

In [1]:
win = tk.Tk() # 창 생성
win.geometry("1000x500") # 창 크기
win.title("exercise") # 제목
win.option_add("*Font", "맑은고딕 25") # 전체 폰트

label = ttk.Label(win, text="Hi")
label.pack()

btn = tk.Button(win) # 버튼 생성
btn.config(width=10, height=10) # 버튼 크기
btn.config(text='quit')
btn.config(command=quit)
btn.pack()

win.mainloop() # 창 And 실행

NameError: name 'tk' is not defined

In [8]:
win = tk.Tk() # 창 생성
win.geometry("3000x1000") # 창 크기
win.title("exercise") # 제목
win.option_add("*Font", "맑은고딕 25") # 전체 폰트

btn = tk.Button(win) # 버튼 생성
btn.config(width=10, height=10) # 버튼 크기

def download():
    folder_selected = filedialog.askdirectory()
    new_df.to_excel(f'{folder_selected}/예산내역서 내용.xlsx', index=False)


def upload():
    filepath = filedialog.askopenfilename(initialdir="", 
                                          title="Select a file",
                                          filetypes=(("csv files", "*.csv"), ("xlsx files", "*.xlsx")))
    original = pd.read_csv(filepath)

    df = original.copy()
    df['영수증'] = get_id(df['영수증 (사진 파일로 업로드)'])
    df['개수'] = [len(v) for v in df['영수증']]
    df = df.drop('영수증 (사진 파일로 업로드)', axis=1)

    real = sum(df['개수'][df['실물영수증 여부']=='실물영수증 O'])

    df = df[['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수']]

    new_df = pd.DataFrame(columns=['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수'])

    index = 0
    for idx, val in df.iterrows():
        num = val['개수']
        for i in range(num):
            new_df.loc[index] = val
            index +=1
            
    btn.config(text='파일 처리가 완료되었습니다. 파일 저장 경로를 선택해주세요.')
    btn.config(command=download)

btn.config(text='구글폼에서 내려받은 파일을 업로드해주세요.') # 버튼 제목
btn.config(command=upload) # 버튼 기능
btn.pack()

win.mainloop() # 창 And 실행

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/shinyehjin/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-8-645802f2c6de>", line 11, in download
    new_df.to_excel(f'{folder_selected}/예산내역서 내용.xlsx', index=False)
NameError: name 'new_df' is not defined


In [ ]:
filepath = filedialog.askopenfilename(initialdir="", 
                                          title="Select a file",
                                          filetypes=(("csv files", "*.csv"), ("xlsx files", "*.xlsx")))

In [21]:
df = pd.read_excel('/Users/shinyehjin/Downloads/예산내역서 내용.xlsx')
list(df.iloc[1])

['동주사진전', 11, '소모품 구입', '상품 구입 / 과자 세트', '2021-05-17', 12900, 'ISP_노아유통', 3]

In [5]:
############################
# 파일 업로드 부분 (csv, xlsx) #
############################

df = original.copy()
df['영수증'] = get_id(df['영수증 (사진 파일로 업로드)'])
df['개수'] = [len(v) for v in df['영수증']]
df = df.drop('영수증 (사진 파일로 업로드)', axis=1)

real = sum(df['개수'][df['실물영수증 여부']=='실물영수증 O'])

df = df[['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수']]

new_df = pd.DataFrame(columns=['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수'])

index = 0
for idx, val in df.iterrows():
    num = val['개수']
    for i in range(num):
        new_df.loc[index] = val
        index +=1
   
############################
# 파일 저장 부분 (path 필요)   #
############################
new_df.to_excel(f'result/use/{month}월_{when}반 예산내역서 내용.xlsx', index=False)

whole_num = sum(df['개수'])
print('영수증은 총 {}개 입니다.'.format(whole_num))
print('실물영수증은 총 {}개 입니다.'.format(real))
print('예산내역서 내용이 저장되었습니다.')

월:  2
전 or 후:  2


FileNotFoundError: [Errno 2] No such file or directory: 'original/use/2월_2반.csv'

In [6]:
# 불러오기
df = original.copy()

# 필요한 column만 남기기
df = df[['사용 RA','인원','결제일자','프로그램명','금액','품명/수량/금액','영수증 (사진 파일로 업로드)']]

# id만 list로 뽑아내는 과정
df['영수증'] = get_id(df['영수증 (사진 파일로 업로드)'])
df = df.drop('영수증 (사진 파일로 업로드)', axis=1)

# 원하는 꼴로 변형
df['인원'] = [str(v)+" 명" for v in df['인원']]
df['금액'] = [str(commaParse(v))+" 원" for v in df['금액']]

# 미리보기
df.head()

NameError: name 'original' is not defined

In [7]:
location_short = [(1640, 205), (2600, 205), (690,270), (1600, 270),(2555, 270)]
location_long = [(1640, 205), (2600, 205), (690,270), (1550, 270),(2555, 270)]
location_house = (690, 205)
location_receipt = (300,670)
location_reason = (550,525)
col_item = [2260, 2690, 2782]
row_item = [685, 830, 964, 1107, 1265, 1402, 1540, 1677, 1830, 1962]

In [16]:
# path 설정
# path = '/Users/shinyehjin/Programming/ydj-accounting-system'

# 폰트 경로와 사이즈 설정
regularFont =ImageFont.truetype('fonts/HANBatangB.ttf',36)
smallFont =ImageFont.truetype('fonts/HANBatangB.ttf',30)

OSError: cannot open resource

In [ ]:
check = 0
for index in range(len(df)):
    
    target_image = Image.open(path+"/original/form.jpeg")
    row = df.iloc[index]

    draw = ImageDraw.Draw(target_image)
    draw.text(location_house, "윤동주 하우스", fill="black", font=regularFont, align='center')
    draw.text(location_reason, str(row[7]), fill="black", font=regularFont, align='center')
    if (len(row[3]) > 9):
        for i in range(5):
            draw.text(location_long[i], str(row[i]), fill="black", font=regularFont, align='center')
    else:
        for i in range(5):
            draw.text(location_short[i], str(row[i]), fill="black", font=regularFont, align='center')
    
    if type(row[5]) == type('string'):
        item_list = divide(row[5])
        for i in range(len(item_list)):
            if len(item_list[i][0]) > 13:
                front = item_list[i][0][:13]
                back = item_list[i][0][13:]
                draw.text((col_item[0], row_item[i]-20), front, fill="black", font=smallFont, align='center') 
                draw.text((col_item[0], row_item[i]+20), back, fill="black", font=smallFont, align='center') 
            else:
                draw.text((col_item[0], row_item[i]), item_list[i][0], fill="black", font=smallFont, align='center') 
            for n in range(1,3):
                draw.text((col_item[n], row_item[i]), item_list[i][n], fill="black", font=smallFont, align='center') 

    for i in range(len(row[6])):
        check += 1
        file_id = row[6][i]
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = googleapiclient.http.MediaIoBaseDownload(fh, request)

        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("[{}] index:{}의 {}번째 사진! ".format(check, index, i)+"Download %d%%." % int(status.progress() * 100))

        add_image = Image.open(fh)
    
        if int(add_image.size[0]*(1300/add_image.size[1])) > 1300:
            target_image.paste(im = add_image.resize((1300, int(add_image.size[1]*(1300/add_image.size[0])))), box=location_receipt)              
        else:
            target_image.paste(im = add_image.resize((int(add_image.size[0]*(1300/add_image.size[1])), 1300)), box=location_receipt)
        target_image.save(path+"/result/use/{}월 {}반/영수증_{}_{}.jpg".format(month, when, index, i))
   
print(f'전체 영수증 개수: {whole_num}\n영수증 양식 제작 개수: {check}')

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import io
import googleapiclient.http

from PIL import Image,ImageDraw,ImageFont
import os

import numpy as np
import pandas as pd
import re

import img2pdf

import tkinter as tk
from tkinter import ttk
from datetime import datetime
from tkinter import filedialog
import time

from Receipt import *

win = tk.Tk() # 창 생성
win.geometry("700x300") # 창 크기
win.title("exercise") # 제목
win.option_add("*Font", "맑은고딕 15") # 전체 폰트

introduce = "1. 아래의 파일 업로드 버튼을 클릭하여 구글폼을 통해 다운 받은 파일 (csv, excel)을 업로드해주세요. \n2. 파일 업로드가 완료되었다면, 영수증 파일과 예산사용내역서를 다운 받을 폴더를 선택해주세요. \n   이때, 폴더 안에 이전에 프로그램을 다운 받았던 파일이 있으면 자동으로 덮어씌워지니 폴더를 잘 선택해주세요.\n4. quit 버튼을 눌러 종료해주세요.\n \n"

label_text = tk.StringVar()
label_text.set(introduce)

label = ttk.Label(win, textvariable=label_text)
label.pack()

progress = tk.DoubleVar()
progress_bar = ttk.Progressbar(win, maximum=100, length=200, variable=progress)
progress_bar.pack()


def start():
	for i in range(100):
		time.sleep(0.1)
		progress.set(i+1)
		progress_bar.update()

btn2 = tk.Button(win) # 버튼 생성
btn2.config(width=10, height=1) # 버튼 크기
btn2.config(text='upload')
btn2.config(command=start)
btn2.pack()


btn = tk.Button(win) # 버튼 생성
btn.config(width=10, height=1) # 버튼 크기
btn.config(text='quit')
btn.config(command=quit)
btn.pack()

win.mainloop() # 창 And 실행











